In [141]:
import os
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12.0, 4.0)
import glob

In [142]:
pd.set_option('display.max_colwidth', -1)

# Datos generales 

In [143]:
file  = open('data/book_data.csv', 'r')  
spanish_names = file.readlines()
spanish_names = [s[:-1] for s in spanish_names]

In [144]:
df = pd.read_csv('data/book_data.csv', delimiter=';')
df.columns = ['Nro', 'Autor', 'Datos_Autor', 'Título', 'Tipo']
df = df.fillna('-----')

styles = [
    dict(selector="td", props=[("text-align", "left")]),
    dict(selector="th", props=[("text-align", "left")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]
html = (df.style.set_table_styles(styles))
html


,Nro,Autor,Datos_Autor,Título,Tipo
0,1,-----,-----,Introducción conjunta a las dos relatorías de la Comisión Histórica del Conflicto y sus Víctimas,introducción
1,2,Eduardo Pizarro Leongómez,-----,Una lectura múltiple y pluralista de la historia,relatoría
2,3,Víctor Manuel Moncayo Cruz,Profesor Emérito Universidad Nacional de Colombia,Hacia la verdad del conflicto: Insurgencia guerrillera y orden social vigente,relatoría
3,4,Sergio De Zubiría Samper,"Profesor Asociado del Departamento de Filosofía. Universidad de los Andes (Bogotá, Colombia)",Dimensiones políticas y culturales en el conflicto colombiano,ensayo
4,5,Gustavo Duncan,Universidad EAFIT y Universidad De los Andes,"Exclusión, Insurrección y Crimen",ensayo
5,6,Jairo Estrada Álvarez,Profesor del Departamento de Ciencia Política Universidad Nacional de Colombia,"Acumulación capitalista, dominación de clase y rebelión armada. Elementos para una interpretación histórica del conflicto social y armado",ensayo
6,7,Javier Giraldo S.J,-----,"Aportes sobre el origen del conflicto armado en Colombia, su persistencia y sus Impactos",ensayo
7,8,Darío Fajardo M,Universidad Externado de Colombia,"Estudio sobre los orígenes del conflicto social armado, razones de su persistencia y sus efectos más profundos en la sociedad colombiana",ensayo
8,9,Jorge Giraldo Ramírez,Doctor en Filosofía y Decano de la Escuela de Ciencias y Humanidades de la Universidad EAFIT,Política y guerra sin compasión,ensayo
9,10,Francisco Gutiérrez Sanín,Investigador del Instituto de Estudios Políticos y Relaciones Internacionales de la Universidad Nacional de Colombia,¿Una historia simple?,ensayo


In [145]:
stop_words_spanish = ['un', 'una', 'unas', 'unos', 'uno', 'sobre', 'todo', 'también', 'tras', 'otro', 'algún', 'alguno', 'alguna', 'algunos', 'algunas', 'ser', 'es', 'soy', 'eres', 'somos', 'sois', 'estoy', 'esta', 'estamos', 'estais', 'estan', 'como', 'en', 'para', 'atras', 'porque', 'por qué', 'estado', 'estaba', 'ante', 'antes', 'siendo', 'ambos', 'pero', 'por', 'poder', 'puede', 'puedo', 'podemos', 'podeis', 'pueden', 'fui', 'fue', 'fuimos', 'fueron', 'hacer', 'hago', 'hace', 'hacemos', 'haceis', 'hacen', 'cada', 'fin', 'incluso', 'primero', 'desde', 'conseguir', 'consigo', 'consigue', 'consigues', 'conseguimos', 'consiguen', 'ir', 'voy', 'va', 'vamos', 'vais', 'van', 'vaya', 'gueno', 'ha', 'tener', 'tengo', 'tiene', 'tenemos', 'teneis', 'tienen', 'el', 'la', 'lo', 'las', 'los', 'su', 'aqui', 'mio', 'tuyo', 'ellos', 'ellas', 'nos', 'nosotros', 'vosotros', 'vosotras', 'si', 'dentro', 'solo', 'solamente', 'saber', 'sabes', 'sabe', 'sabemos', 'sabeis', 'saben', 'ultimo', 'largo', 'bastante', 'haces', 'muchos', 'aquellos', 'aquellas', 'sus', 'entonces', 'tiempo', 'verdad', 'verdadero', 'verdadera', 'cierto', 'ciertos', 'cierta', 'ciertas', 'intentar', 'intento', 'intenta', 'intentas', 'intentamos', 'intentais', 'intentan', 'dos', 'bajo', 'arriba', 'encima', 'usar', 'uso', 'usas', 'usa', 'usamos', 'usais', 'usan', 'emplear', 'empleo', 'empleas', 'emplean', 'ampleamos', 'empleais', 'valor', 'muy', 'era', 'eras', 'eramos', 'eran', 'modo', 'bien', 'cual', 'cuando', 'donde', 'mientras', 'quien', 'con', 'entre', 'sin', 'trabajo', 'trabajar', 'trabajas', 'trabaja', 'trabajamos', 'trabajais', 'trabajan', 'podria', 'podrias', 'podriamos', 'podrian', 'podriais', 'yo', 'aquel', 'a', 'en', 'de', 'para', 'por', 'según', 'sin', 'sobre', 'tras', 'bajo', 'contra', 'hacia', 'hasta', 'rt', 'que', 'se', 'no', 'del', 'al', 'http', 'https', 'y', 'más', 'todos', 'año', 'será', 'este', 'vía', 'les', 'ni', 'dice', 'le', 'está', 'qué', 'hay', 'htt', 'quiere', 'nuestra', 'sí', 'of', 'in', 'años', 'firma' , 'firman', 'nuevo', 'hoy', 'ayer', 'inicio', 'ya', 'histórico', 'día','son', 'final', 'and', 'with', 'you', 'mundo', 'gran', 'sol', 'recibe', 'millones',  'gracias', 'hermana', 'fuego', 'me', 'eso',  'we', 'after', 'camino', 'dijo', 'caminos', 'recibirá', 'entiendo', 'mejores', 'mejor', 'deseos', 'lleve', 'nuevamente', 'ahora', 'definitiva', 'ese', 'definitivo', 'han', 'sino', 'tal', 'mismos', 'estos', 'así', 'ver', 'esto', 'esta', 'sido', 'manera', 'sigo', 'sin embargo', '0 0', 'op cit', 'cit p', 'de la' ]

file  = open('data/spanish_names.txt', 'r')  
spanish_names = file.readlines()
spanish_names = [s[:-1] for s in spanish_names]

abc = string.ascii_lowercase 
punct_signs = string.punctuation

In [146]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_top_words = 20
n_samples = 2000
n_features = 1000
n_topics = 5
n_top_words = 10

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d: " % topic_idx,  " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

from nltk import ngrams
import operator
from itertools import islice
import string 
from tabulate import tabulate

def stopWordsInGrams(grams):    
    count = 0
    for w in grams:
        if w in stop_words_spanish:
            count = count + 1                
    return count/len(grams)  

def isNumber(s):
    try: 
        int(s)
        float(s)
        return True
    except ValueError:
        return False

def getNGram(text, n, stop_words, m):
    ngramas = ngrams(text.split(), n)
    h_dict = {}   
    for grams in ngramas:
        if stopWordsInGrams(grams) > 0.34:
            continue
        words = ' '.join(grams)        
        words = words.lower() 
        words = ''.join(e for e in words if e.isalnum() or e == ' ')    
        words = words.strip()  
        if  words in stop_words or words in punct_signs or words == '' or words in abc or isNumber(words) or 'rt' in words or 'http' in words or 'https' in words or len(words.split(' ')) != n:
             continue
        if words in h_dict:
            h_dict[words] = h_dict[words] + 1
        else: 
            h_dict[words] = 1 
    sorted_dict = sorted(h_dict.items(), key=operator.itemgetter(1), reverse=True)
    return list(islice(sorted_dict, m))

def fetch_files_from(path):
    files = []    
    for filename in glob.glob(os.path.join(path, '*.txt')):
        with open(filename) as f:
            files.append(f.read())
    return files

def printTopicsLDAbyCat(dataset):
    data_samples = dataset[:n_samples]        

    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1,
                                           max_features=n_features,
                                           stop_words=stop_words_spanish)

    tfidf = tfidf_vectorizer.fit_transform(data_samples)
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=1,
                                    max_features=n_features,
                                    stop_words=stop_words_spanish)

    tf = tf_vectorizer.fit_transform(data_samples)

    lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)

    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_top_words)

## 5 Topics por Documento  LDA from Scikit

In [147]:
for index, row in df.iterrows():      
    print ( row['Autor'], ', ', row['Título'], '[', row['Tipo'],']')
    print()
    filepath = 'data/libro/' + str(index+1) + '.txt'
    print(filepath)
    file = open(filepath)
    text = file.read()    
    text = [text[:50], text[51:100], text[100:]]
    printTopicsLDAbyCat(text)
    print()

----- ,  Introducción conjunta  a las dos  relatorías de la Comisión  Histórica  del Conflicto y sus Víctimas [ introducción ]

data/libro/1.txt
Topic #0:  colombia nacional conflicto social bogotá parte política violencia guerra político
Topic #1:  colombia violencia bogotá guerra social política conflicto nacional político estados
Topic #2:  colombia conflicto bogotá nacional guerra violencia política parte social sociales
Topic #3:  colombia conflicto guerra violencia política bogotá nacional social parte país
Topic #4:  colombia nacional bogotá guerra política social violencia conflicto sociales político


Eduardo Pizarro Leongómez ,  Una lectura múltiple y pluralista de la historia [ relatoría ]

data/libro/2.txt
Topic #0:  violencia conflicto colombia grupos nacional país frente guerra pécaut político
Topic #1:  conflicto colombia grupos armado violencia nacional período jorge pécaut guerra
Topic #2:  conflicto colombia violencia nacional grupos armado política país giraldo pobla


data/libro/15.txt
Topic #0:  nacional guerra bogotá colombia historia ejército política anexos violencia congreso
Topic #1:  bogotá colombia guerra nacional política historia violencia congreso social liberales
Topic #2:  colombia nacional guerra bogotá política historia ejército políticas social militares
Topic #3:  bogotá colombia guerra política conservadores social frente nacional documento guerrillas
Topic #4:  nacional congreso cultura 49 1991 bogotá población 2014 hecho política


----- ,  Comunicado Conjunto,  5 de agosto  de 2014 [ comunicado ]

data/libro/16.txt
Topic #0:  soportes reflejar incluyendo ep adjuntar duración expertos harán comisión general
Topic #1:  expertos difusión conversaciones ningún histórica general mecanismos debe víctimas duración
Topic #2:  chcv víctimas comunicado informe comisión optar mesa selección farc delegación
Topic #3:  expertos comisión informe informes conflicto síntesis mesa debe mandato relatores
Topic #4:  expertos informes informe rela

## Top 20 Unigramas por documento

In [148]:
for index, row in df.iterrows():      
    print ( row['Autor'], ', ', row['Título'], '[', row['Tipo'],']')
    print()
    filepath = 'data/libro/' + str(index+1) + '.txt'
    print(filepath)
    file = open(filepath)
    data = file.read()    
    final = getNGram(data, 1, stop_words_spanish, 20)
    print(tabulate(final, ['Ngram', 'Number']))
    print()

----- ,  Introducción conjunta  a las dos  relatorías de la Comisión  Histórica  del Conflicto y sus Víctimas [ introducción ]

data/libro/1.txt
Ngram         Number
----------  --------
colombia        1187
conflicto        891
guerra           787
bogotá           736
violencia        734
política         707
nacional         691
social           602
país             436
sociales         403
estados          392
gobierno         374
político         368
guerrillas       353
unidos           348
frente           345
políticas        323
militar          316
población        314
durante          311

Eduardo Pizarro Leongómez ,  Una lectura múltiple y pluralista de la historia [ relatoría ]

data/libro/2.txt
Ngram        Number
---------  --------
conflicto       159
colombia        148
violencia       143
nacional        114
grupos           95
armado           90
país             70
política         70
giraldo          63
población        61
guerra           60
frente           57
ta

Ngram                Number
-----------------  --------
estados                 266
colombia                266
unidos                  265
bogotá                  119
militar                 101
guerra                   91
militares                77
the                      71
país                     64
ejército                 63
fuerzas                  60
nacional                 56
asistencia               53
contrainsurgencia        52
colombiano               48
seguridad                44
plan                     42
gobierno                 41
cia                      39
violencia                39

María Emma Wills Obregón ,  Los tres nudos de la guerra colombiana [ ensayo ]

data/libro/15.txt
Ngram            Number
-------------  --------
bogotá               86
colombia             77
guerra               70
nacional             67
política             53
violencia            36
social               34
historia             34
congreso             33
frente               3

## Top 20 Bigramas por documento

In [149]:
for index, row in df.iterrows():      
    print ( row['Autor'], ', ', row['Título'], '[', row['Tipo'],']')
    print()
    filepath = 'data/libro/' + str(index+1) + '.txt'
    print(filepath)
    file = open(filepath)
    data = file.read()    
    final = getNGram(data, 2, stop_words_spanish, 20)
    print(tabulate(final, ['Ngram', 'Number']))
    print()

----- ,  Introducción conjunta  a las dos  relatorías de la Comisión  Histórica  del Conflicto y sus Víctimas [ introducción ]

data/libro/1.txt
Ngram                   Number
--------------------  --------
estados unidos             337
conflicto armado           185
frente nacional            175
colombia bogotá            121
siglo xx                   102
fuerzas armadas             97
américa latina              96
conflicto social            95
grupos paramilitares        82
la violencia                77
universidad nacional        77
reforma agraria             74
derechos humanos            72
grupos armados              66
francisco gutiérrez         65
bogotá universidad          65
población civil             64
fuerza pública              63
jorge giraldo               62
orden social                59

Eduardo Pizarro Leongómez ,  Una lectura múltiple y pluralista de la historia [ relatoría ]

data/libro/2.txt
Ngram                   Number
--------------------  --------


Ngram                   Number
--------------------  --------
ee uu                       24
el gobierno                 17
reforma agraria             12
fuerza pública              10
el liberalismo               9
magdalena medio              9
la violencia                 9
lópez pumarejo               9
lleras restrepo              8
rojas pinilla                8
el ejército                  8
movimiento campesino         8
el general                   7
lucha armada                 7
el davis                     7
mil días                     7
grupos armados               6
el espectador                6
a fines                      6
laureano gómez               6

Daniel Pécaut ,  Una lucha armada al servicio del statu quo social y político [ ensayo ]

data/libro/12.txt
Ngram                   Number
--------------------  --------
conflicto armado            26
frente nacional             20
lucha armada                13
fuerza pública              12
las guerrillas         

## Top 35 Unigramas del Libro Completo 

In [150]:
abc = string.ascii_lowercase 
punct_signs = string.punctuation
data = ""
for index, row in df.iterrows():      
    filepath = 'data/libro/' + str(index+1) + '.txt'
    file = open(filepath)
    data = data + '\n' + file.read()    
final = getNGram(data, 1, stop_words_spanish, 35)
print(tabulate(final, ['Ngram', 'Number']))
print()

Ngram            Number
-------------  --------
colombia           2368
conflicto          1765
guerra             1572
bogotá             1471
violencia          1466
política           1413
nacional           1380
social             1201
país                872
sociales            806
estados             783
gobierno            747
político            735
guerrillas          706
unidos              695
frente              690
políticas           645
militar             632
población           627
durante             622
grupos              604
armado              598
farc                592
orden               582
fuerzas             565
siglo               556
colombiano          554
sociedad            544
políticos           544
seguridad           542
tanto               534
proceso             530
desarrollo          522
paramilitares       506
tierra              498



## Top 35 Bigramas del Libro Completo 

In [151]:
abc = string.ascii_lowercase 
punct_signs = string.punctuation
data = ""
for index, row in df.iterrows():      
    filepath = 'data/libro/' + str(index+1) + '.txt'
    file = open(filepath)
    data = data + '\n' + file.read()    
final = getNGram(data, 2, stop_words_spanish, 35)
print(tabulate(final, ['Ngram', 'Number']))
print()

Ngram                   Number
--------------------  --------
estados unidos             673
conflicto armado           365
frente nacional            350
colombia bogotá            242
siglo xx                   204
fuerzas armadas            193
américa latina             192
conflicto social           189
grupos paramilitares       164
la violencia               154
universidad nacional       154
reforma agraria            148
derechos humanos           144
grupos armados             132
bogotá universidad         130
francisco gutiérrez        129
población civil            128
fuerza pública             126
jorge giraldo              123
orden social               117
sistema político           114
memoria histórica          114
university press           110
conflicto colombiano       107
la guerra                  102
daniel pécaut              101
sociedad colombiana        101
estado colombiano          100
fuerzas militares          100
guerra civil               100
plan col

## Top 35 Nombres Propios del Libro Completo

In [152]:
import re   
import operator
from collections import Counter

def extractNamesAndEntities(text):
    i = 0
    hdict = {}
    patron = re.compile('(([A-ZÁÉÍÓÚÑÜ]{1}[a-zñáéíóúü]+)([ ]+de[ ]+la[ ]+)?([ ]+de[ ]+)?([ ]+del[ ]+)?([ ]*[A-ZÁÉÍÓÚÑÜ]{1}[a-zñáéíóúü]+)*)')
    names = patron.findall(text)
    names = [x[0] for x in names if x[0].lower() not in stop_words_spanish]
    for k in names:
        k = k.replace('Según', '')
        k = k.replace('Informes', '')
        k = k.replace('Informe', '')
        k = k.strip()
        for n in spanish_names:            
            if(n is not '' and (n + ' ' in k.upper() or ' ' + n in k.upper() or n == k.upper()) ):       
                if(k in hdict.keys()):
                    hdict[k] = hdict[k] + 1
                else:
                    hdict[k] = 1
                break
    return hdict

def printTopMNames(hdict, m):    
    sorted_dict = sorted(hdict.items(), key=operator.itemgetter(1), reverse=True)
    return list(islice(sorted_dict, m))

In [153]:
data = ''
hdict = {}
for index, row in df.iterrows():
    filepath = 'data/libro/' + str(index+1) + '.txt'
    file = open(filepath)
    data = data + '\n' + file.read()    

hdict = extractNamesAndEntities(data)
final = printTopMNames(hdict, 35)
print(tabulate(final, ['Nombre', 'Cantidad']))
print()

Nombre                     Cantidad
-----------------------  ----------
Jorge Giraldo                   126
Gaitán                          122
Francisco Gutiérrez             116
Daniel Pécaut                    92
Alfredo Molano                   76
María Emma Wills                 62
Paz                              58
Darío Fajardo                    54
Magdalena                        52
Magdalena Medio                  48
Carlos                           48
Javier Giraldo                   40
Pablo Escobar                    38
Gustavo Duncan                   36
Francisco                        35
Álvaro                           32
Vicente Torrijos                 32
David                            30
Norma                            30
Análisis Político                30
Franco                           30
Mauricio                         30
Eduardo                          29
Alfonso López Michelsen          28
Álvaro Uribe                     28
Jacobo Arenas               

## Top 20 Nombres Propios más mencionados por categoría

In [154]:
for index, row in df.iterrows():
    print ( row['Autor'], ', ', row['Título'], '[', row['Tipo'],']')
    print()
    filepath = 'data/libro/' + str(index+1) + '.txt'
    file = open(filepath)
    data = file.read()    
    hdict = extractNamesAndEntities(data)
    final = printTopMNames(hdict, 20)
    print(tabulate(final, ['Nombre', 'Cantidad']))
    print()

----- ,  Introducción conjunta  a las dos  relatorías de la Comisión  Histórica  del Conflicto y sus Víctimas [ introducción ]

Nombre                 Cantidad
-------------------  ----------
Jorge Giraldo                63
Gaitán                       61
Francisco Gutiérrez          58
Daniel Pécaut                46
Alfredo Molano               38
María Emma Wills             31
Paz                          29
Darío Fajardo                27
Magdalena                    26
Magdalena Medio              24
Carlos                       24
Javier Giraldo               20
Pablo Escobar                19
Gustavo Duncan               18
Francisco                    18
Álvaro                       16
Vicente Torrijos             16
David                        15
Norma                        15
Eduardo                      15

Eduardo Pizarro Leongómez ,  Una lectura múltiple y pluralista de la historia [ relatoría ]

Nombre                     Cantidad
-----------------------  ----------
Fr

Nombre                       Cantidad
-------------------------  ----------
Francisco Gutiérrez                 9
Elisabeth Jean Wood                 3
Paz                                 3
Alejandro Reyes                     3
Jorge                               3
Carlos Miguel Ortiz                 3
Carlos Lleras Restrepo              2
Elisabeth Wood                      2
Carlos Castaño                      2
Teófilo Vásquez                     2
Análisis Político                   2
Francisco Gutiérrez Sanín           2
Norma                               2
William Ramírez Tobón               2
Daniel Pécaut                       2
Editorial Norma                     2
Magdalena                           2
Rocío Peña                          2
César Caballero                     2
Hernán Giraldo                      2

Afredo Molano Bravo ,  Fragmentos de la historia del conflicto armado (1920-2010) [ ensayo ]

Nombre                    Cantidad
----------------------  ----------